In [64]:
from Functions.get_word_pairs import get_wp_in_line_hard, get_vectorized_wp_and_wp
from Functions.get_types_of_event import get_types_of_event_spacy
from Functions.get_wp_distances import get_distances_between_wp
import tensorflow as tf
import numpy as np
import spacy
from keras.models import load_model
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors


In [14]:
nlp_classic = spacy.load('ru_core_news_sm')
nlp_type_of_event_extraction = spacy.load('../../../PipeLines/WordsExtraction/words_extract/')
kcm_extraction_model = load_model('../../../PipeLines/Coder/coder_from_spacy_to_kcm_onh5.h5')
is_description_model = load_model('../../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize.h5')

In [15]:
text = "ТПП Омской области проводит обучающие мероприятия для предпринимателей-руководителей организаций :\nИнтерактивный практикум по оценке рисков предпринимательской деятельности « Три красных сигнала» \nНа практикуме вы узнаете:\n» Как определить степень вероятности кризиса в компании;\n» Какие показатели хозяйственной деятельности предприятия необходимо анализировать;\n» Как разработать план антикризисных мероприятии;\n» Как разделить ответственность между собственником и наемным руководителем;\n» На что обращать внимание при заключении контрактов.\nУчастие в мероприятии бесплатное!"

In [16]:
V = list()
N = list()

In [17]:
for sentence in text.split('.')[:-1]:
    wp_of_event, wp_of_event_vectorized = get_vectorized_wp_and_wp(word_pairs=get_wp_in_line_hard(sentence), 
                                                                   vectorizer=lambda text: nlp_classic(text).vector)
    is_wp_descriptor = is_description_model.predict(wp_of_event_vectorized) > 0.992
    V.append(wp_of_event_vectorized[is_wp_descriptor.T[0]])
    N.append(wp_of_event[is_wp_descriptor.T[0]])

50/50 [==============================] - 0s 1ms/step


In [61]:
V[0].std()

0.7961713

In [91]:
dbscan = DBSCAN(eps=0.3, min_samples=2, metric='cosine').fit(V[0])

In [92]:
np.unique(dbscan.labels_)

array([0, 1], dtype=int64)

In [93]:
for i, j in zip(N[0], dbscan.labels_):
    print(i, j)

практикум по оценке рисков предпринимательской 0
практикум по оценке рисков предпринимательской деятельности 0
практикум по оценке рисков предпринимательской деятельности « 0
по оценке рисков предпринимательской деятельности 0
оценке рисков 0
оценке рисков предпринимательской 0
оценке рисков предпринимательской деятельности 0
оценке рисков предпринимательской деятельности « 0
рисков предпринимательской 0
рисков предпринимательской деятельности 0
рисков предпринимательской деятельности « 0
предпринимательской деятельности 0
предпринимательской деятельности « 0
Как определить степень вероятности 1
Как определить степень вероятности кризиса 1
Как определить степень вероятности кризиса в 1
определить степень 1
определить степень вероятности 1
определить степень вероятности кризиса 1
определить степень вероятности кризиса в 1
определить степень вероятности кризиса в компании;
» Какие показатели хозяйственной деятельности 0
определить степень вероятности кризиса в компании;
» Какие показател

In [55]:
kmeans = KMeans(2, n_init='auto').fit(V[0])

In [36]:
np.unique(kmeans.labels_)

array([0, 1])

In [23]:
for i, j in zip(N[0], kmeans.labels_):
    print(i, j)

практикум по оценке рисков предпринимательской 1
практикум по оценке рисков предпринимательской деятельности 1
практикум по оценке рисков предпринимательской деятельности « 1
по оценке рисков предпринимательской деятельности 1
оценке рисков 1
оценке рисков предпринимательской 1
оценке рисков предпринимательской деятельности 1
оценке рисков предпринимательской деятельности « 1
рисков предпринимательской 1
рисков предпринимательской деятельности 1
рисков предпринимательской деятельности « 1
предпринимательской деятельности 1
предпринимательской деятельности « 1
Как определить степень вероятности 0
Как определить степень вероятности кризиса 0
Как определить степень вероятности кризиса в 0
определить степень 0
определить степень вероятности 0
определить степень вероятности кризиса 0
определить степень вероятности кризиса в 0
определить степень вероятности кризиса в компании;
» Какие показатели хозяйственной деятельности 1
определить степень вероятности кризиса в компании;
» Какие показател

In [74]:
nn = NearestNeighbors(n_neighbors=5, radius=0.79, metric='cosine').fit(V[0])
distances, indices = nn.kneighbors(V[0])

In [75]:
[c[np.argsort(d)[0]] for c, d in zip(indices, distances)]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51]

In [76]:
for i, j in zip(N[0], nn):
    print(i, j)

AttributeError: 'NearestNeighbors' object has no attribute 'labels_'